### Priority Queue with Message Passing (Go)

A priority queue is used for serving requests of higher priority quicker than those of lower priority. For example, priority queues are used in [Microsoft's Azure cloud](https://docs.microsoft.com/en-us/azure/architecture/patterns/priority-queue) in "applications that offer different service level guarantees to individual clients." The task is to implement a bounded priority queue in Go similar to a bounded buffer, except that a priority is attached to incoming messages. Priorities are *absolute*, meaning that messages with higher priority always take precedence. Consequently, messages of lower priority may potentially be neglected forever. Messages of the same priority must be served first-in, first-out. The priority is a non-negative number, with `0` being the highest priority. Messages are simply strings. The capacity of the priority queue is specified when the `priorityQueue` goroutine is created. Complete `priorityQueue` below!
- If the priority queue is empty, only a message from `west` can be received.
- If the priority queue is full, only a message to `east` can be sent.
- If the priority queue is neither full nor empty, a message from `west` can be received or a message to `east` sent, whatever is requested.

The function `main` contains two tests. You can organize the incoming messages in the buffer in any way as long as insertion and deletion take time linear in the size of the buffer.

In [1]:
%%writefile priorityqueue.go
package main
import ("math/rand"; "strconv"; "sort")

type PriorityMessage struct {
    Priority int // non-negative
    Message string
}

func priorityQueue(capacity int, west chan PriorityMessage, east chan string) {
    var pq []PriorityMessage

    for{
        if len(pq) == 0 {
                fromWest := <- west
                pq = append(pq, fromWest)
        } else if len(pq) == capacity {
            east <- pq[0].Message
            pq = pq[1:]
        } else {
            select{

                //Resource: https://stackoverflow.com/questions/46128016/insert-a-value-in-a-slice-at-a-given-index
                case fromWest := <- west:
                    pos := 0
                    for pos < len(pq) && fromWest.Priority >= pq[pos].Priority {pos++}
                    if pos == len(pq) { 
                        pq = append(pq, fromWest) 
                    } else {
                        pq = append(pq[:pos+1], pq[pos:]...)
                        pq[pos] = fromWest
                    }

                case east <- pq[0].Message:
                    pq = pq[1:]
            }

        }
    }
}

func sendMessages(n int, ch0 chan PriorityMessage, ch1 chan string) { // 0 <= n <= 90, number of messages
    for s := 10; s < n + 10; s++ { // 2-digit serial number
        prio := rand.Intn(10) // 1-digit priority
        m := strconv.Itoa(prio) + "." + strconv.Itoa(s)
        ch0 <- PriorityMessage{prio, m}; ch1 <- m
    }
}
func main() {
    const C, R = 20, 10 // capacity, rounds of testing
    west := make(chan PriorityMessage)
    south, east := make(chan string),  make(chan string)
    
    go priorityQueue(C, west, east)
    
    // testing priority queue exactly at capacity: received messages must be sent messages in ascending order
    for t := 0; t < 10; t++ {
        var in, out [C]string
        go sendMessages(C, west, south) // priority queue is filled up
        for i := 0; i < C; i++ { // messages sent to priority queue are copied to array in
            in[i] = <- south
        }
        for i := 0; i < C; i++ { // messages received from priority queue are stored in array out 
            out[i] = <- east; print(out[i], " ") // printed in ascending order
        }
        sort.Strings(in[:]) // sort the sent messages
        if in != out {panic("received messages must be sent messages in ascending order")}
        println()
    }
    
    // testing with more messages than capacity: received messages may not always be in ascending order
    for t := 0; t < 10; t++ {
        var in, out [2 * C]string
        go sendMessages(2 * C, west, south) // priority queue is filled up
        go func () {
            for i := 0; i < 2 * C; i++ { // messages sent to priority queue are copied to array in
                in[i] = <- south
            }
        }()
        for i := 0; i < 2 * C; i++ { // messages received from priority queue are stored in array out 
            out[i] = <- east; print(out[i], " ") // printed in not necessarily ascending order
        }
        sort.Strings(in[:]); sort.Strings(out[:]) // sort the sent and received messages
        if in != out {panic("all sent messages must be received (in some order)")}
        println()
    }
}

Overwriting priorityqueue.go


In [2]:
!go run priorityqueue.go

0.21 0.25 2.20 3.10 3.11 3.13 4.15 4.28 5.14 5.17 6.18 7.22 7.26 7.27 7.29 8.23 8.24 9.12 9.16 9.19 
0.10 0.29 1.27 2.25 2.26 3.17 4.12 4.15 4.20 4.21 4.23 4.28 5.18 6.16 6.24 8.11 8.13 8.19 9.14 9.22 
0.13 0.15 0.22 1.24 2.10 2.14 2.19 2.21 2.27 3.11 4.26 4.29 7.16 7.20 7.23 8.25 9.12 9.17 9.18 9.28 
0.16 0.20 0.24 0.25 0.29 1.13 2.10 4.18 4.21 4.28 5.17 5.19 5.22 5.27 6.12 6.14 8.15 8.23 8.26 9.11 
2.16 2.21 2.26 2.27 3.20 3.23 3.29 5.14 6.17 6.19 7.11 7.24 8.12 8.28 9.10 9.13 9.15 9.18 9.22 9.25 
0.15 1.10 1.23 2.27 3.12 4.20 5.16 5.18 5.24 6.14 6.17 6.19 7.11 7.21 7.25 7.29 8.22 8.28 9.13 9.26 
0.13 0.24 0.25 0.27 1.16 2.10 2.18 3.14 4.21 5.15 5.20 6.11 7.17 7.26 7.28 8.22 8.29 9.12 9.19 9.23 
0.12 0.18 0.27 1.10 1.11 1.17 1.21 1.24 3.14 3.20 4.19 4.26 5.15 5.29 6.13 6.23 7.16 7.22 8.25 8.28 
0.19 2.10 2.20 3.14 4.13 4.26 4.29 5.18 6.16 6.17 6.24 6.27 7.11 7.12 7.15 7.21 7.22 7.25 8.23 8.28 
0.11 1.13 2.12 2.27 4.14 4.15 5.10 6.21 7.16 7.20 7.24 7.28 8.19 8.26 9.17 9.18 9.22 9.23 9

*Bonus.*  Use a heap to implement the priority queue, such that insertion and deletion are guaranteed to take time logarithmic in the number of messages in the priority queue. You can use textbooks or other resources to implement the heap, but you must cite all resources. Note that heaps are unstable in that the order of elements with the same key is not preserved. Here, the order of messages with the same priority should be preserved. To make a heap stable, a *timestamp*, implemented by an integer counter, can be added to messages and used for comparison. [2 bonus points]

In [ ]:
%%writefile priorityqueue.go
package main
import ("math/rand"; "strconv"; "sort")

type PriorityMessage struct {
    Priority int // non-negative
    Message string
}

func priorityQueue(capacity int, west chan PriorityMessage, east chan string) {
    # YOUR CODE HERE
    raise NotImplementedError()
}
func sendMessages(n int, ch0 chan PriorityMessage, ch1 chan string) { // 0 <= n <= 90, number of messages
    for s := 10; s < n + 10; s++ { // 2-digit serial number
        prio := rand.Intn(10) // 1-digit priority
        m := strconv.Itoa(prio) + "." + strconv.Itoa(s)
        ch0 <- PriorityMessage{prio, m}; ch1 <- m
    }
}
func main() {
    const C, R = 20, 10 // capacity, rounds of testing
    west := make(chan PriorityMessage)
    south, east := make(chan string),  make(chan string)
    
    go priorityQueue(C, west, east)
    
    // testing priority queue exactly at capacity: received messages must be sent messages in ascending order
    for t := 0; t < 10; t++ {
        var in, out [C]string
        go sendMessages(C, west, south) // priority queue is filled up
        for i := 0; i < C; i++ { // messages sent to priority queue are copied to array in
            in[i] = <- south
        }
        for i := 0; i < C; i++ { // messages received from priority queue are stored in array out 
            out[i] = <- east; print(out[i], " ") // printed in ascending order
        }
        sort.Strings(in[:]) // sort the sent messages
        if in != out {panic("received messages must be sent messages in ascending order")}
        println()
    }
    
    // testing with more messages than capacity: received messages may not always be in ascending order
    for t := 0; t < 10; t++ {
        var in, out [2 * C]string
        go sendMessages(2 * C, west, south) // priority queue is filled up
        go func () {
            for i := 0; i < 2 * C; i++ { // messages sent to priority queue are copied to array in
                in[i] = <- south
            }
        }()
        for i := 0; i < 2 * C; i++ { // messages received from priority queue are stored in array out 
            out[i] = <- east; print(out[i], " ") // printed in not necessarily ascending order
        }
        sort.Strings(in[:]); sort.Strings(out[:]) // sort the sent and received messages
        if in != out {panic("all sent messages must be received (in some order)")}
        println()
    }
}

In [ ]:
!go run priorityqueue.go